# Milestone Project 2 - Blackjack Game
In this milestone project you will be creating a Complete BlackJack Card Game in Python.

Here are the requirements:

* You need to create a simple text-based [BlackJack](https://en.wikipedia.org/wiki/Blackjack) game
* The game needs to have one player versus an automated dealer.
* The player can stand or hit.
* The player must be able to pick their betting amount.
* You need to keep track of the player's total money.
* You need to alert the player of wins, losses, or busts, etc...

And most importantly:

* **You must use OOP and classes in some portion of your game. You can not just use functions in your game. Use classes to help you define the Deck and the Player's hand. There are many right ways to do this, so explore it well!**


Feel free to expand this game. Try including multiple players. Try adding in Double-Down and card splits! Remember to you are free to use any resources you want and as always:

# HAVE FUN!

In [3]:
from random import randint
from random import shuffle
from IPython.display import clear_output

class Actor:
    def __init__(self,name):
        self.cards = []
        self.score = 0
        self.name = name
#     --------------------------
    def pick_card(self,deck,flag=True):
        shuffle(deck)
        rand_card = deck.pop()
        val = 0
        if rand_card[0][0] == 'A':
            if flag and self.decide() == 11:
                val = 11
            else:
                val = 1
        else:
            val = rand_card[1]
        
        if not flag:
            self.cards += '#'
        else:
            self.cards.append(rand_card[0])
            self.score += val
#     --------------------------    
    def decide(self):
        pass
#     --------------------------
    def display_game(self):
        print(f'> {self.name}: ')
        print(f'    cards: {self.cards}')
        print(f'    score: {self.score}')
#     --------------------------
    def refresh(self):
        self.cards = []
        self.score = 0
#     --------------------------
#     --------------------------
class Player(Actor):
    def __init__(self,deck,cash):
        Actor.__init__(self,'Player')
        self.cash = cash
        self.refresh()
#     --------------------------
    def decide(self):
        message = 'You have Axe => 11 or 1?: '
        while True:
            try:
                val = int(input(message))
                if val != 1 and val != 11:
                    continue
            except:
                message = 'Int requered.'
            else:
                return val
#     --------------------------    
    def make_bet(self):
        bet = 0
        message = "$$$ Make a bet: "
        while True:
            try:
                bet = input(message)
                bet = int(bet)
                if bet > self.cash:
                    message = f'Bet should be less than your cash({self.cash})'
                    continue
                if bet < 1:
                    message = f'Bet should be a positive integer({self.cash})'
                    continue
            except Exception as ex:
#                 template = "An exception of type {0} occurred. Arguments:\n{1!r}"
#                 m = template.format(type(ex).__name__, ex.args)
#                 print(m)
                message = f"Bet shoud be an integer (you entered '{bet}'): "
                continue
            else:
                self.cash -= bet
                break
        return bet
#     --------------------------
    def play(self,deck):
        message = 'Take or stay? (t/s): '
        display_stat = True
        while True:
            if display_stat:
                self.display_game()
            move = input(message).lower()
            if move != 't' and move != 's':
                message = "Enter 's' or 't': "
                display_stat = False
                continue
            display_stat = True
            if move == 't':
                message = 'Take or stay? (t/s): '
                self.pick_card(deck)
                if self.score > 21:
                    self.display_game()
                    break
                continue
            break
#     --------------------------
    def display_game(self):
        Actor.display_game(self)
        print(f'    cash: {self.cash}')
#     --------------------------
    def refresh(self):
        Actor.refresh(self)
        self.pick_card(deck)
        self.pick_card(deck)
        self.display_game()
#     --------------------------
#     --------------------------
class Diller(Actor):
    def __init__(self,deck):
        Actor.__init__(self,'Diller')
        self.refresh()
# ----------------------------
    def decide(self):
        if 11 + self.score > 21:
            return 1
        else:
            if randint(1,2) == 1:
                return 11
            return 1
# ----------------------------
    def play(self,deck,pl_score):
        while True:
            self.display_game()
            self.pick_card(deck)
            if self.score > pl_score or self.score > 21:
                self.display_game()
                break
# ----------------------------
    def refresh(self):
        Actor.refresh(self)
        self.pick_card(deck,flag=False)
        self.pick_card(deck)
        self.display_game()
# ----------------------------
# ----------------------------
def deck_init():
    list1 = list(range(2,11))
    list1 += ['J','Q','K','A']
    list2 = ['clubs','harts','spades','diamods']
    res = []
    for x in list1:
        for y in list2:
            if isinstance(x,int):
                res.append(tuple([f'{x} of {y}', x]))
            elif isinstance(x,int) and x == 'A':
                res.append(tuple([f'{x} of {y}', 11, 1]))
            else:
                res.append(tuple([f'{x} of {y}', 10]))
    return res
# ----------------------------
def new_game(deck,player,diller):
    clear_output()
    deck = deck_init()
    player.refresh()
    diller.refresh()
# ----------------------------
def lost_message(bet):
    print('='*21)
    print(f'|=== You lost {bet} ===|')
    print('='*21)
# ----------------------------
def win_message(player,bet):
    print('='*20)
    print(f'|=== You won {bet*2} ===|')
    print('='*20)
    player.cash += bet*2
# ----------------------------
def play_game(deck,player,diller,flag=True):
    if player.cash == 0:
        return False
    message = 'Do you want to play blackjack? (y/n): '
    while True:
        x = input(message).lower()
        if x != 'y' and x != 'n':
            message = "Enter 'y' or 'n': "
            continue
        if x == 'y':
            if flag:
                new_game(deck,player,diller)
            return True
        return False
# ----------------------------
# ----------------------------
# ----------------------------
deck = deck_init()
player = Player(deck,cash=100)
diller = Diller(deck) 
flag = False    
while play_game(deck,player,diller,flag):
    bet = player.make_bet()
    flag = True
    
    player.play(deck)
    if player.score > 21:
        lost_message(bet)
        continue
    if player.score == 21:
        win_message(player,bet)
        continue
    
    diller_score = diller.play(deck,player.score)
    if diller.score > 21:
        win_message(player,bet)
        continue
    if diller.score > player.score:
        lost_message(bet)
        continue

> Player: 
    cards: ['5 of diamods', 'J of clubs']
    score: 15
    cash: 70
> Diller: 
    cards: ['#', '5 of harts']
    score: 5
$$$ Make a bet: 1
> Player: 
    cards: ['5 of diamods', 'J of clubs']
    score: 15
    cash: 69
Take or stay? (t/s): t
> Player: 
    cards: ['5 of diamods', 'J of clubs', '4 of harts']
    score: 19
    cash: 69
Take or stay? (t/s): s
> Diller: 
    cards: ['#', '5 of harts']
    score: 5
> Diller: 
    cards: ['#', '5 of harts', '10 of diamods']
    score: 15
> Diller: 
    cards: ['#', '5 of harts', '10 of diamods', '10 of spades']
    score: 25
|=== You won 2 ===|
Do you want to play blackjack? (y/n): n
